In [1]:
import re
import urllib
import urllib.request
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
%matplotlib inline
plt.style.use('ggplot')
sns.set_style('darkgrid')

In [2]:
# embaralhei a lista de players pra nao pegar só de GM - NAO PRECISA RODAR ESSA CEDULA DNV
# shuffled_players.to_csv('shuffled_player_list_chesscom.csv',index = False)

In [3]:
players = pd.read_csv('shuffled_player_list_chesscom.csv')

In [4]:
players

,username,title,bullet_rating,blitz_rating,rapid_rating
0,michaelq2d5,GM,2793,2765,2903
1,bernd_stromberg,CM,2619,2701,0
2,g2_coder,FM,2350,2200,0
3,mennabarreto,FM,2251,2407,0
4,radojeiriz,FM,2388,2271,2489
...,...,...,...,...,...
8556,skiiier,FM,2492,2477,0
8557,volodymyrmolyboha,NM,2431,2503,2189
8558,miodrag-jevtic,FM,2378,2356,1936
8559,jerryle,NM,2214,2251,1365


In [5]:
# resolve erro "This event loop is already running"
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [6]:
# instala e importa https://github.com/sarartur/chess.com
# wrapper para dados publicados pelo chess.com
!pip install chessdotcom
import chessdotcom

ERROR: Could not find a version that satisfies the requirement chessdotcom
ERROR: No matching distribution found for chessdotcom


In [26]:
# peguei um role do stack overflow e editei, pega os anos/meses que teve jogo de cada pessoa

def getDates(user):
    username = user #change 
    baseUrl = "https://api.chess.com/pub/player/" + username + "/games/"
    archivesUrl = baseUrl + "archives"

    #read the archives url and store in a list
    f = urllib.request.urlopen(archivesUrl)
    archives = f.read().decode("utf-8")
    archives = archives.replace("{\"archives\":[\"", "\",\"")
    archivesList = archives.split("\",\"" + baseUrl)
    archivesList[len(archivesList)-1] = archivesList[len(archivesList)-1].rstrip("\"]}")
    #print(archivesList)
    month = []
    year = []
    if archivesList[0] == '{"archives":[':
        print('Erro - Jogador não possui partidas registradas')
        return(month, year) 
    archivesList.remove('')
    for i in archivesList:
        year.append(int(i[0:4]))
        month.append(int(i[5:]))
    return(month, year)

In [8]:
games_db = pd.read_csv('empty_games.csv')
games_db.head()

,id,ranked,time_control,status,winner,white,black,moves


In [9]:
def checkstatus(white_result, black_result):
    if white_result != 'win':
        return white_result
    else:
        return black_result
    
def checkwinner(white, black):
    if white['result'] == 'win':
        return white['username']
    elif black['result'] == 'win':
        return black['username']
    else:
        return 'draw'

def trataPGN(pgn):
    # remove clock
    txt = '{\[%clk \d+:\d+:\d+(\.\d+)?\]}'
    a = re.sub(txt, "", pgn)

    # remove header
    txt = '\[.+\]'
    a = re.sub(txt, "", a)
    a

    # remove \n
    txt = '\\n'
    a = re.sub(txt, "", a)
    a
    
    return(a)    

In [27]:
# escolhe que jogadores voce quer pegar
players_to_db = players[20:25]
players_to_db.head()

,username,title,bullet_rating,blitz_rating,rapid_rating
20,king_kinto,CM,2150,2219,2099
21,sidarun,NM,2248,2237,2177
22,tittanok,GM,0,0,0
23,taxticsw2,FM,2678,0,0
24,jovencata,CM,2414,2540,2045


In [28]:
games_db = pd.read_csv('empty_games.csv')

for k in players_to_db['username']:
    month, year = getDates(k)
    print(k)
    for i in range(len(month)):
        response = chessdotcom.get_player_games_by_month(k,month=month[i],year=year[i]).json
        for j in (response['games']):
            try:
                temp = {
                    'id':j['uuid'],
                    'ranked':j['rated'],
                    'time_control': j['time_class'],
                    'status': checkstatus(j['white']['result'],j['black']['result']),
                    'winner': checkwinner(j['white'],j['black']),
                    'white': (j['white']['username']),
                    'black': (j['black']['username']),
                    'moves': trataPGN(j['pgn'])
                }
                #print((j['white']['result'],j['black']['result']))
                games_db = games_db.append(temp,ignore_index=True)
            except:
                temp = 0

king_kinto
sidarun
Erro - Jogador não possui partidas registradas
tittanok
taxticsw2
jovencata


In [29]:
games_db

,id,ranked,time_control,status,winner,white,black,moves
0,e851f078-3af7-11dd-8000-000000010001,True,blitz,resigned,king_kinto,king_kinto,chesspartner123,1. d4 1... Nf6 2. c4 2... g6 3. Nc3 3... ...
1,1a635200-3af8-11dd-8000-000000010001,True,blitz,resigned,chesspartner123,chesspartner123,king_kinto,1. e4 1... d6 2. d4 2... Nf6 3. Nc3 3... ...
2,54ccef1e-3af8-11dd-8000-000000010001,True,blitz,resigned,king_kinto,king_kinto,chesspartner123,1. e4 1... d5 2. exd5 2... Nf6 3. d4 3......
3,7a59f8ee-3af8-11dd-8000-000000010001,True,blitz,resigned,king_kinto,chesspartner123,king_kinto,1. e4 1... d6 2. d3 2... e5 3. g3 3... g5...
4,9362a99e-3af8-11dd-8000-000000010001,True,blitz,resigned,chesspartner123,king_kinto,chesspartner123,1. d4 1... Nf6 2. c4 2... c5 3. d5 3... b...
...,...,...,...,...,...,...,...,...
10585,32af8c9c-2782-11ec-8605-09ff3c010001,True,bullet,resigned,robertxbob,robertxbob,jovencata,1. d4 1... a6 2. c4 2... b5 3. cxb5 3... ...
10586,5cbdb155-2782-11ec-8605-09ff3c010001,True,bullet,resigned,jovencata,jovencata,robertxbob,1. Nf3 1... g6 2. g3 2... Bg7 3. Bg2 3......
10587,d5ee8e31-27d9-11ec-8605-09ff3c010001,True,bullet,resigned,jovencata,robertxbob,jovencata,1. d4 1... a6 2. c4 2... b5 3. cxb5 3... ...
10588,c9e83b37-417d-11ec-90b8-09ff3c010001,True,bullet,timeout,Jr_teng,Jr_teng,jovencata,1. e4 1... e6 2. d4 2... d5 3. Nc3 3... d...


In [32]:
# salva em csv
games_db.to_csv('games_db_21_25.csv',index = False)

In [ ]:
# Pesquisa de nomes no dataframe
#w = 'chesspartner123'
#texts = [*filter(lambda x: w in x, games_db['winner'])]
#print(*texts, sep=f'\n\n{"=" * 200}\n\n')